# SMAC Counterfactual Analysis Demo

This notebook demonstrates running counterfactual analysis on SMAC with a random policy.

In [1]:
import sys
import os
import logging
import absl.logging
from contextlib import contextmanager
# Get the absolute path to the agents folder
examples_path = os.path.abspath('../')
smac_agents_path = os.path.abspath('../../playing-with-smac/smac/agents')
ppo_path = os.path.join(smac_agents_path, 'PPO_one_action')

sys.path.insert(0, smac_agents_path)  # For utils
sys.path.insert(0, ppo_path)          # For random_policy
sys.path.insert(0, examples_path)     # For utils
import sys
sys.path.insert(0, '/home/ad.msoe.edu/manchadoa/UR-RL/counterfactual-reasoning/src')
os.environ['SC2PATH'] = '/home/ad.msoe.edu/manchadoa/UR-RL/playing-with-smac/StarCraftII'


print(f"Added: {smac_agents_path}")
print(f"Added: {ppo_path}")

Added: /home/ad.msoe.edu/manchadoa/UR-RL/playing-with-smac/smac/agents
Added: /home/ad.msoe.edu/manchadoa/UR-RL/playing-with-smac/smac/agents/PPO_one_action


## 1. Setup SMAC Environment

In [2]:
from smac.env import StarCraft2Env
from counterfactual_rl.environments.smac import CentralizedSmacWrapper, SmacStateManager

# Create SMAC environment (3m = 3 Marines vs 3 Marines)
smac_env = StarCraft2Env(map_name="3m", debug=True)
# Wrap it for centralized control
env = CentralizedSmacWrapper(smac_env, use_state=True)

# Get environment info
print(f"Agents: {env.n_agents}")
print(f"Actions per agent: {env.n_actions_per_agent}")
print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Version: B55958 (SC2.3.16)
Build: Jul 31 2017 13:19:41
Command Line: '"/home/ad.msoe.edu/manchadoa/UR-RL/playing-with-smac/StarCraftII/Versions/Base55958/SC2_x64" -listen 127.0.0.1 -port 41005 -dataDir /home/ad.msoe.edu/manchadoa/UR-RL/playing-with-smac/StarCraftII/ -tempDir /tmp/sc-twluc3_h/'
Starting up...
Startup Phase 1 complete
Startup Phase 2 complete
Creating stub renderer...
Startup Phase 2 complete
Creating stub renderer...
Listening on: 127.0.0.1:41005 (41005)
Startup Phase 3 complete. Ready for commands.
Listening on: 127.0.0.1:41005 (41005)
Startup Phase 3 complete. Ready for commands.
Requesting to join a single player game
Configuring interface options
Configure: raw interface enabled
Configure: feature layer interface disabled
Configure: score interface disabled
Configure: render interface disabled
Entering load game phase.
Launching next game.
Next launch phase started: 2
Next launch phase started: 3
Next launch phase started: 4
Next launch phase started: 5
Next launch 

Agents: 3
Actions per agent: 9
Action space: MultiDiscrete([9 9 9])
Observation space: Box(-inf, inf, (48,), float32)


Game has started.
Sending ResponseJoinGame


## 2. Create Random Policy

In [3]:
from random_policy import RandomPolicy

# Create random policy (uses raw SMAC env for action masking)
policy = RandomPolicy(env=smac_env)

print(f"Policy n_agents: {policy.n_agents}")
print(f"Policy n_actions: {policy.n_actions}")

Policy n_agents: 3
Policy n_actions: 9


## 3. Setup Counterfactual Analyzer

In [ ]:
from counterfactual_rl.analysis import MultiDiscreteCounterfactualAnalyzer
from utils import get_valid_actions

# Create analyzer
analyzer = MultiDiscreteCounterfactualAnalyzer(
    model=policy,
    env=env,
    state_manager=SmacStateManager,
    get_valid_actions_fn=lambda: get_valid_actions(smac_env),
    get_action_probs_fn=None,  # Uniform probability for random policy
    n_agents=policy.n_agents,
    n_actions=policy.n_actions,
    horizon=10,
    n_rollouts=10,  # Keep low for demo
    top_k=2,
    deterministic=False,
    log_file="smac_counterfactual_analysis.log",
)


print("Analyzer created!")
print(f"Logging to: smac_counterfactual_analysis.log")

/home/ad.msoe.edu/manchadoa/.conda/envs/counterfactual/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:CounterfactualAnalyzer_135326330639696:================================================================================
INFO:CounterfactualAnalyzer_135326330639696:Counterfactual Analysis Started
INFO:CounterfactualAnalyzer_135326330639696:Configuration: n_agents=3, n_actions=9
INFO:CounterfactualAnalyzer_135326330639696:Parameters: horizon=10, n_rollouts=10, top_k=2
INFO:CounterfactualAnalyzer_135326330639696:Gamma=0.99, Deterministic=False
INFO:CounterfactualAnalyzer_135326330639696:================================================================================
INFO:CounterfactualAnalyzer_135326330639696:================================================================================
INFO:Count

Analyzer created!
Logging to: smac_counterfactual_analysis.log


## 4. Run Counterfactual Analysis

In [5]:
import numpy as np

In [6]:
avail_actions_all = np.array([[0, 1, 1, 1, 1, 1, 0, 0, 0], [0, 1, 1, 1, 1, 1, 0, 0, 0], [0, 1, 1, 1, 1, 1, 0, 0, 0]])

print(avail_actions_all)
available_actions = [np.where(agent_avail_actions == 1)[0].tolist() for agent_avail_actions in avail_actions_all]

print(available_actions)


[[0 1 1 1 1 1 0 0 0]
 [0 1 1 1 1 1 0 0 0]
 [0 1 1 1 1 1 0 0 0]]
[[1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5]]


In [7]:
# Evaluate a single episode
print("Running counterfactual analysis...")

records = analyzer.evaluate_episode(max_steps=2, verbose=True)

print(f"\nCollected {len(records)} records")

Version: B55958 (SC2.3.16)
Build: Jul 31 2017 13:19:41
Command Line: '"/home/ad.msoe.edu/manchadoa/UR-RL/playing-with-smac/StarCraftII/Versions/Base55958/SC2_x64" -listen 127.0.0.1 -port 35915 -dataDir /home/ad.msoe.edu/manchadoa/UR-RL/playing-with-smac/StarCraftII/ -tempDir /tmp/sc-l0_qs4re/'
Starting up...
Startup Phase 1 complete


Running counterfactual analysis...


Startup Phase 2 complete
Creating stub renderer...
Listening on: 127.0.0.1:35915 (35915)
Startup Phase 3 complete. Ready for commands.
Listening on: 127.0.0.1:35915 (35915)
Startup Phase 3 complete. Ready for commands.
RequestQuit command received.
Closing Application...
unable to parse websocket frame.
Requesting to join a single player game
Configuring interface options
Configure: raw interface enabled
Configure: feature layer interface disabled
Configure: score interface disabled
Configure: render interface disabled
Entering load game phase.
Launching next game.
Next launch phase started: 2
Next launch phase started: 3
Next launch phase started: 4
Next launch phase started: 5
Next launch phase started: 6
Next launch phase started: 7
Next launch phase started: 8
RequestQuit command received.
Closing Application...
unable to parse websocket frame.
Requesting to join a single player game
Configuring interface options
Configure: raw interface enabled
Configure: feature layer interface d


Collected 2 records


## 5. Analyze Results

In [8]:
# Show consequence scores
if records:
    scores = [r.consequence_score for r in records]
    print(f"Mean consequence score: {np.mean(scores):.4f}")
    print(f"Max consequence score: {np.max(scores):.4f}")
    print(f"Min consequence score: {np.min(scores):.4f}")
    
    # Show most consequential step
    most_consequential = max(records, key=lambda r: r.consequence_score)
    print(f"\nMost consequential step:")
    print(f"  Action: {most_consequential.action}")
    print(f"  Score: {most_consequential.consequence_score:.4f}")

Mean consequence score: 0.0000
Max consequence score: 0.0000
Min consequence score: 0.0000

Most consequential step:
  Action: (np.int64(3), np.int64(1), np.int64(2))
  Score: 0.0000


In [9]:
# Cleanup
smac_env.close()
print("Environment closed.")

RequestQuit command received.
Closing Application...
unable to parse websocket frame.


Environment closed.
